In [1]:
from collections import defaultdict
def convertVE2EV(sampled_path,file_name,max_edges):
    # 
    degrees = defaultdict(int)
    edges = list()
    
    print('Step 1 to Bipartite Graph')
     # Step 1 to Bipartite Graph
    bipartite_edges = dict() # eVertex_id,pair
    vertex_at_first = defaultdict(list)
    vertex_at_second = defaultdict(list)
    vertex_label = dict()
    undirect_vertex = defaultdict(list)
    
    in_path = '%s/%s.labeled.%s' % (sampled_path,file_name,'sampled')
    print('reading form: %s' % in_path)
    index = 0
    with open(in_path, 'r', encoding='utf-8') as fin:
        lines = fin.readlines()
        print(lines[0])
        num = int(lines[0][0:-1])
        for line in lines[1:]:
            if index % 1000000 == 0:
                print('progress %d/%d ' % (index,len(lines)))
            index += 1
            items = line.strip('\n').split('\t')

            vertex1 = str(items[0])
            vertex2 = str(items[1])
            weight = float(items[2]) # VG-Edge val
            labels1 = set([int(x) for x in items[3].split(' ')])
            labels2 = set([int(x) for x in items[4].split(' ')])
            overlap_ratio = float(items[5])
            
            degrees[vertex1] += 1
            degrees[vertex2] += 1
            
            vertex_label[vertex1] = labels1
            vertex_label[vertex2] = labels2
            
            label_and_set = labels1 & labels2
            label_or_set = labels1 | labels2
            
            # undirected graph
            edges.append((vertex1, vertex2,label_and_set,label_or_set, 1.0))
            
    
    # All mode | Or mode
    # eVertex == edge-centric Vertex
    eVertex_labelA = dict()
    eVertex_labelO = dict()
    for edge in edges:
        (vertex1, vertex2,labelsA,labelsO,weight) = edge
        eVertex_id = '(%s->%s)' % (edge[0],edge[1])
        eVertex_labelA[eVertex_id] = labelsA
        eVertex_labelO[eVertex_id] = labelsO
        bipartite_edges[eVertex_id] = edge
        vertex_at_first[vertex1].append(eVertex_id)
        vertex_at_second[vertex2].append(eVertex_id)
        undirect_vertex[vertex1].append(eVertex_id)
        undirect_vertex[vertex2].append(eVertex_id)
    
    print('Step 2 to Edge-Centic Graph')
    
    # Step 2 to Edge-Centic Graph
    e_vertex_counter = defaultdict(float)
    ec_edges = []
    ec_edges_or = []
    
    # filter out isolate vertices
    exist_links = set()
    def check(id1,id2):
        if id1 < id2:
            idx = id1+' '+id2
        else:
            idx = id2+' '+id1
        if idx in exist_links:
            return False
        else:
            exist_links.add(idx)
            return True
    
    for index ,e_vertex in enumerate(bipartite_edges):
        if index % 100000 == 0:
            print('progress %d/%d ' % (index,len(bipartite_edges)))
        (vertex1, vertex2, weight,labelsA,labelsO) = bipartite_edges[e_vertex]

        # eVertex1 / eVertex2 maay linked by 1st or 2nd vertex
        first_neighbors = vertex_at_first[vertex1] + vertex_at_second[vertex1] 
        second_neighbors = vertex_at_second[vertex2] + vertex_at_first[vertex2]
        for neighbor_id in set(first_neighbors):
            if neighbor_id != e_vertex and check(neighbor_id,e_vertex):
                # first type of label
                label1_str = ' '.join([str(x) for x in eVertex_labelA[e_vertex]])
                label2_str = ' '.join([str(x) for x in eVertex_labelA[neighbor_id]])
                try:
                    overlap_ratio = len(eVertex_labelA[e_vertex] & eVertex_labelA[neighbor_id]) / len(eVertex_labelA[e_vertex] | eVertex_labelA[neighbor_id])   
                except ZeroDivisionError:
                    overlap_ratio = 0
                if overlap_ratio > 0: 
                    ec_edges.append((e_vertex, neighbor_id, degrees[vertex1],label1_str,label2_str,overlap_ratio))
                # second type
                label1_str = ' '.join([str(x) for x in eVertex_labelO[e_vertex]])
                label2_str = ' '.join([str(x) for x in eVertex_labelO[neighbor_id]])
                try:
                    overlap_ratio = len(eVertex_labelO[e_vertex] & eVertex_labelO[neighbor_id]) / len(eVertex_labelO[e_vertex] | eVertex_labelO[neighbor_id])   
                except ZeroDivisionError:
                    overlap_ratio = 0
                ec_edges_or.append((e_vertex, neighbor_id, degrees[vertex1],label1_str,label2_str,overlap_ratio))
                if overlap_ratio > 0: 
                    e_vertex_counter[e_vertex] += degrees[vertex1]
        for neighbor_id in set(second_neighbors):
            if neighbor_id != e_vertex and  check(neighbor_id,e_vertex):
                label1_str = ' '.join([str(x) for x in eVertex_labelA[e_vertex]])
                label2_str = ' '.join([str(x) for x in eVertex_labelA[neighbor_id]])
                try:
                    overlap_ratio = len(eVertex_labelA[e_vertex] & eVertex_labelA[neighbor_id]) / len(eVertex_labelA[e_vertex] | eVertex_labelA[neighbor_id])   
                except ZeroDivisionError:
                    overlap_ratio = 0
                if overlap_ratio > 0:  
                    ec_edges.append((e_vertex, neighbor_id, degrees[vertex2],label1_str,label2_str,overlap_ratio))
                
                label1_str = ' '.join([str(x) for x in eVertex_labelO[e_vertex]])
                label2_str = ' '.join([str(x) for x in eVertex_labelO[neighbor_id]])
                try:
                    overlap_ratio = len(eVertex_labelO[e_vertex] & eVertex_labelO[neighbor_id]) / len(eVertex_labelO[e_vertex] | eVertex_labelO[neighbor_id])   
                except ZeroDivisionError:
                    overlap_ratio = 0
                if overlap_ratio > 0:    
                    ec_edges_or.append((e_vertex, neighbor_id, degrees[vertex2],label1_str,label2_str,overlap_ratio))
                    e_vertex_counter[e_vertex] += degrees[vertex2]
                
        label1_str = ' '.join([str(x) for x in eVertex_labelA[e_vertex]])
        label2_str = ' '.join([str(x) for x in eVertex_labelA[e_vertex]])
        overlap_ratio = 1.0
        ec_edges.append((e_vertex, e_vertex, degrees[vertex1] + degrees[vertex2], label1_str,label2_str,overlap_ratio))

        
        label1_str = ' '.join([str(x) for x in eVertex_labelO[e_vertex]])
        label2_str = ' '.join([str(x) for x in eVertex_labelO[e_vertex]])
        overlap_ratio =1.0
        ec_edges_or.append((e_vertex, e_vertex, degrees[vertex1] + degrees[vertex2],label1_str,label2_str,overlap_ratio))
        
        e_vertex_counter[e_vertex] += degrees[vertex1]
        e_vertex_counter[e_vertex] += degrees[vertex2]
    
    # sort by overlap
    ec_edges=sorted(ec_edges, key=lambda x:x[-1], reverse=True)
    ec_edges_or=sorted(ec_edges_or, key=lambda x:x[-1], reverse=True)
    print('original_edges: %d' % len(ec_edges_or))
    if max_edges != -1:
        ec_edges = ec_edges[0:max_edges]
        ec_edges_or = ec_edges_or[0:max_edges]
    # print(bipartite_edges)
    # Step3 to File
    import os
    with open(sampled_path+'/eca_'+file_name+'.labeled.sampled','w+') as fout:
        fout.write('%d\n' % (len(ec_edges)))
        for i,edge in enumerate(ec_edges):
            fout.write('%s\t%s\t%.2f\t%s\t%s\t%f\n' % edge)
        print('eca_edges: %d' % len(ec_edges))
    with open(sampled_path+'/eco_'+file_name+'.labeled.sampled','w+') as fout:
        fout.write('%d\n' % (len(ec_edges_or)))
        for i,edge in enumerate(ec_edges_or):
            fout.write('%s\t%s\t%.2f\t%s\t%s\t%f\n' % edge)
        print('eco_edges: %d' % len(ec_edges_or))
    
    print('########done' )
    
        
 
convertVE2EV('generated_data','youtube.txt', max_edges = -1)    

Step 1 to Bipartite Graph
reading form: generated_data/youtube.txt.labeled.sampled
10000

progress 0/10001 
Step 2 to Edge-Centic Graph
progress 0/10000 
original_edges: 278740
eca_edges: 89662
eco_edges: 278740
########done


In [10]:
sorted(['G-Y','Y-G'])

['G-Y', 'Y-G']